In [17]:
from dotenv import load_dotenv
from azure.ai.inference.aio import EmbeddingsClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.aio import SearchClient
from models.resolution import Resolution
from azure.search.documents.models import VectorizedQuery
from dataclasses import asdict
from openai import AsyncAzureOpenAI
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from typing import List
import os
import json

In [5]:
load_dotenv(override=True)

search_endpoint = os.getenv('SEARCH_ENDPOINT')
search_api_key = os.getenv('SEARCH_API_KEY')

endpoint=os.getenv('AI_FOUNDRY_OPENAI_ENDPOINT')
api_key = os.getenv('OPENAI_KEY')
chat_model = os.getenv('OPEN_AI_CHAT_MODEL')

cohere_key = os.getenv('COHERE_KEY')
cohere_model=os.getenv('COHERE_MODEL')
cohere_endpoint=os.getenv('COHERE_ENDPOINT')



In [20]:
embedding_client = EmbeddingsClient(
    endpoint=cohere_endpoint,
    credential=AzureKeyCredential(cohere_key)
)
model_name = cohere_model    

search_client = SearchClient(
    endpoint=search_endpoint,
    index_name="cartroubleshooting",
    credential=AzureKeyCredential(search_api_key)
)    

chat_client = AsyncAzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint=endpoint,
    api_key=api_key,
)

In [ ]:
response = await chat_client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "I am going to Paris, what should I see?",
        }
    ],
    max_tokens=4096,
    temperature=1.0,
    top_p=1.0,
    model=chat_model
)

print(response.choices[0].message.content)

In [18]:
template_instruction = """You are a helpful car troubleshooting assistant. Your role is to help users find resolutions to car problems in multiple languages.

You will be provided with relevant troubleshooting information retrieved from a database based on the user's question. This information is included below:

{context}

Your responsibilities:

1. DETECT the language of the user's question and remember it for your response.

2. CRITICAL - Language Handling:
   - ALWAYS respond to the user in the SAME language they used in their question
   - Supported languages include: English, Spanish, French, German, Italian, Portuguese, Dutch, Russian, Chinese, Japanese, Korean, Arabic, and many others
   - Even if the provided troubleshooting information is in a different language, translate your response to match the user's language
   - Keep the technical terms (brand, model) as provided, but translate your explanations and instructions to the user's language

3. Present the troubleshooting solutions ONLY using the information provided in the context above. You MUST NOT add any additional troubleshooting advice, suggestions, or information that is not explicitly provided in the context.

4. The provided context contains Resolution information with:
   - id: The resolution ID
   - score: Relevance score (higher is better match)
   - brand: Car brand
   - model: Car model
   - fault: Description of the fault
   - fix: How to fix the issue

5. Format your response based ONLY on the data provided in the context:
   - Present each solution clearly with the car brand and model
   - Show the fault description (translated to user's language if needed)
   - Provide the fix instructions (translated to user's language if needed)
   - You may mention the relevance score to help users understand how well it matches their problem
   - Do NOT supplement with general automotive knowledge
   - Do NOT make assumptions or add information not present in the provided context

6. Keep responses concise and practical:
   - Focus on the specific fixes provided
   - Present all solutions provided in the context (typically top 3 matches)
   - End with encouraging words in the user's language such as:
     * English: "Try these fixes and see if they resolve your issue"
     * Spanish: "Prueba estas soluciones y verifica si resuelven tu problema"
     * French: "Essayez ces solutions et voyez si elles résolvent votre problème"
     * (Adapt to other languages as needed)

7. If the context is empty or contains no troubleshooting information, politely inform the user in their language that no matching solutions were found for their problem.

Remember:
- ALWAYS respond in the SAME language the user used in their question
- ONLY present information that is provided in the context
- Keep the tone friendly and helpful while staying factual to the provided data
- If the user's query is completely unrelated to car problems, politely explain in their language that you can only help with car troubleshooting issues"""

In [ ]:
async def get_answer_from_model(prompt:str, context:List[Resolution]) -> str:
    resolutions_json = json.dumps([asdict(res) for res in context], indent=2)
    system_msg = template_instruction.replace('{context}',resolutions_json)
    response = await chat_client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system_msg
            },
            {
                "role": "user",
                "content": prompt,
            }
        ],
        max_tokens=4096,
        temperature=1.0,
        top_p=1.0,
        model=chat_model
    )

    print(response.choices[0].message.content)    

Create the plugin to get embedding from the prompt

In [15]:
async def embed( prompt:str) -> List[Resolution]:
    """
    Get the vector embedding representation of a text prompt.
    
    Args:
        prompt (str): The text prompt to convert into a vector embedding
        
    Returns:
        List[Resolution]: A list of Resolution objects containing the top matching
                         troubleshooting solutions, including car brand, model,
                         fault description, and fix instructions
    """
    response = await embedding_client.embed(input=[prompt],
                                            model=cohere_model)
    
    vector = response.data[0]['embedding']

    vector_query = VectorizedQuery(
        vector=vector,
        k_nearest_neighbors=3,
        fields="vector"
    )

    results = await search_client.search(
        search_text="",
        vector_queries=[vector_query],
        select=["id","brand","model","fault","fix"]                
    )

    resolutions:List[Resolution] = []
    #resolutions = []
    async for result in results:
        #resolutions.append(result)
        resolutions.append(Resolution(
            id=result['id'],
            score=result['@search.score'],
            brand=result['brand'],
            model=result['model'],
            fault=result['fault'],
            fix=result['fix']
        ))

    return resolutions


In [ ]:
results = await embed("Battery draining fast")

print(results)

In [14]:
question = "Mon moteur est beaucoup trop chaud"

context = await embed(question)